# TSP Nearest Insertion Heuristic

So we have a set of 8 cities or vertex $V = \{1,2,3,4,5,6,7,8\}$ with a corresponding $D = d_{i,j} i, j \in V$ distance matrix between every city

In [1]:
V = collect(1:8)

8-element Vector{Int64}:
 1
 2
 3
 4
 5
 6
 7
 8

In [2]:
D = 
[0 89 87 38 33 71 59 54
89 0 32 59 65 39 45 61
87 32 0 50 75 17 64 79
38 59 50 0 40 33 50 56
33 65 75 40 0 62 26 21
71 39 17 33 62 0 57 70
59 45 64 50 26 57 0 16
54 61 79 56 21 70 16 0]

8×8 Matrix{Int64}:
  0  89  87  38  33  71  59  54
 89   0  32  59  65  39  45  61
 87  32   0  50  75  17  64  79
 38  59  50   0  40  33  50  56
 33  65  75  40   0  62  26  21
 71  39  17  33  62   0  57  70
 59  45  64  50  26  57   0  16
 54  61  79  56  21  70  16   0

And we will have a $T$ set which will represent the ordered tour of our TSP

In [3]:
T = Int64[]

Int64[]

And all of this minimizing the distance between cities, while connecting every single one of them once.

To start, let's pick the smallest tour available, by getting the minimal distance in the matrix. Now, because I am lazy I will use the `findmin` function defined in Julia but it will return the zero elements of the main diagonal. So what I will do first is to replace all of those with undesirable distances. A naive approach would be to simply do a `replace` but it will mess with other 0s in the matrix, so let´s stick to the diagonal. I could also use the LinearAlgebra stdlib but I wanna try stuff by myself

In [4]:
for i ∈ 1:8
    D[i, i] = 999999
end
D

8×8 Matrix{Int64}:
 999999      89      87      38      33      71      59      54
     89  999999      32      59      65      39      45      61
     87      32  999999      50      75      17      64      79
     38      59      50  999999      40      33      50      56
     33      65      75      40  999999      62      26      21
     71      39      17      33      62  999999      57      70
     59      45      64      50      26      57  999999      16
     54      61      79      56      21      70      16  999999

In [5]:
mindis, coords = findmin(D)
x = coords[1]
y = coords[2]
println("Min distance $mindis found between $x and $y cities")

Min distance 16 found between 8 and 7 cities


So we add those to our $T$ tour

In [6]:
push!(T, y)
push!(T, x)

2-element Vector{Int64}:
 7
 8

And we create a $\bar{V}$ without the cities we've already selected. I will create a helper function for that

In [7]:
function remove!(V, item)
    deleteat!(V, findall(x->x==item, V))
end
V̄ = copy(V)
remove!(V̄, x)
remove!(V̄, y)

6-element Vector{Int64}:
 1
 2
 3
 4
 5
 6

I will define a custom struct for keeping track of the cities and their distances to our current $T$

In [8]:
struct city
    distance::Int64
    node::Int64
end

So while $\bar{V}$ is not empty we shall iterate, adding the nearest node to our current tour and selecting the best place for insertion. We can use many metrics to define "nearest node", either the mean or the sum of distances or simply the minimum value. In this case, I will be using the sum of all distances of node $i$ to $T$

In [9]:
citiesDis = city[]
for i in V̄
    dist = 0
    for j in T
        dist = dist + D[i,j]
    end
    cityDis = city(dist, i)
    push!(citiesDis, cityDis)
end
citiesDis

6-element Vector{city}:
 city(113, 1)
 city(106, 2)
 city(143, 3)
 city(106, 4)
 city(47, 5)
 city(127, 6)

Now that we know all of our distances, let´s pick the smallest one available and get the node that is represented by such distance to $T$. To do so, I will use multiple dispatch of the < operator with our new type. Just like operator overloading, but better.

In [10]:
Base.isless(c1::city, c2::city) = c1.distance < c2.distance
optimum = findmin(citiesDis) # this uses the multiple dispatch defined in the previous line.
mindis = optimum[1].distance # Why [1]? because findmin returns a tuple of the CITY and its index.
# right now the index equals the node field but later on it will break, so that's why it's coupled in the struct.
î = optimum[1].node
println("Min distance to T $mindis from city $î")

Min distance to T 47 from city 5


We must determine the best place to insert this node. I am too bored to type the LaTeX in of the equation.

In [11]:
insertion = Tuple[]
for q in 1:length(T)
    if q ≠ length(T)
        distances = D[T[q], î] + D[î,T[q+1]] - D[T[q], T[q+1]]
        insert = tuple(T[q], distances)
        push!(insertion, insert)
    else
        distances = D[T[q], î] + D[î,T[q-1]] - D[T[q], T[q-1]]
        insert = tuple(T[q], distances)
        push!(insertion, insert)
    end
end
sums = [x[2] for x in insertion] # the first [1] field of our tuple represents the node of the insertion, [2] the value.
mindis, idx = findmin(sums)
optCity = insertion[idx][1] #then again, as its an array of tuples we have to access the node with [1].
println("Inserting after city $optCity with a weight of $mindis")

Inserting after city 7 with a weight of 31


In this case it doesn't matter really where do we insert the new node 5, so lets just insert it at 8.

In [12]:
optCityIdx = findfirst(x->x==optCity, T)
insert!(T, optCityIdx+1, î)
remove!(V̄, î)
println("Current tour T: $T")

Current tour T: [7, 5, 8]


So lets create our loop iterating $\bar{V}$

In [13]:
while !isempty(V̄)
    citiesDis = city[]
    for i in V̄
        dist = 0
        for j in T
            dist = dist + D[i,j]
        end
        cityDis = city(dist, i)
        push!(citiesDis, cityDis)
    end
    optimum = findmin(citiesDis)
    mindis = optimum[1].distance 
    î = optimum[1].node
    println("Min distance to T $mindis from city $î")
    insertion = Tuple[]
    for q in 1:length(T)
        if q ≠ length(T)
            distances = D[T[q], î] + D[î,T[q+1]] - D[T[q], T[q+1]]
            insert = tuple(T[q], distances)
            push!(insertion, insert)
        else
            distances = D[T[q], î] + D[î,T[q-1]] - D[T[q], T[q-1]]
            insert = tuple(T[q], distances)
            push!(insertion, insert)
        end
    end
    sums = [x[2] for x in insertion]
    mindis, idx = findmin(sums)
    optCity = insertion[idx][1]
    println("Inserting after city $optCity with a weight of $mindis")
    optCityIdx = findfirst(x->x==optCity, T)
    insert!(T, optCityIdx+1, î)
    remove!(V̄, î)
    println("Current tour T: $T")
end
println("---------End of the heuristic---------")
println("Final tour T: $T")
W = 0
for i in 1:length(T) -1
    W = D[T[i], T[i+1]] + W
end
W = W + D[last(T), first(T)]
println("Total weight W of: $W")

city[city(146, 1), city(171, 2), city(218, 3), city(146, 4), city(189, 6)]
Min distance to T 146 from city 1
Inserting after city 7 with a weight of 66
Current tour T: [7, 1, 5, 8]
city[city(260, 2), city(305, 3), city(184, 4), city(260, 6)]
Min distance to T 184 from city 4
Inserting after city 7 with a weight of 29
Current tour T: [7, 4, 1, 5, 8]
city[city(319, 2), city(355, 3), city(293, 6)]
Min distance to T 293 from city 6
Inserting after city 7 with a weight of 40
Current tour T: [7, 6, 4, 1, 5, 8]
city[city(358, 2), city(372, 3)]
Min distance to T 358 from city 2
Inserting after city 7 with a weight of 27
Current tour T: [7, 2, 6, 4, 1, 5, 8]
city[city(404, 3)]
Min distance to T 404 from city 3
Inserting after city 2 with a weight of 10
Current tour T: [7, 2, 3, 6, 4, 1, 5, 8]
---------End of the heuristic---------
Final tour T: [7, 2, 3, 6, 4, 1, 5, 8]
Total weight W of: 235
